In [1]:
import numpy as np
import open3d as o3d
from tqdm import tqdm_notebook
import os, sys, shutil
import matplotlib.pyplot as plt
import cv2

In [2]:
IMG_FOLDER_PATH = "/home/saby/Projects/ati/data/data/datasets/Carla/64beam-Data/pair_transform_dynseg_all/emptycities_preprocess/test/d_img_40_512/"

In [3]:
def linear_transform(sample_img, A=0, B=255, a=-1, b=1):
    mul_fact = (b-a)/(B-A)
    sub_fact = A
    add_fact = a
    out_img = (sample_img-sub_fact)*mul_fact + add_fact
    return out_img

def to_polar_np(velo):
    if len(velo.shape) == 4:
        velo = velo.transpose(1, 2, 3, 0)

    if velo.shape[2] > 4:
        assert velo.shape[0] <= 4
        velo = velo.transpose(1, 2, 0, 3)
        switch=True
    else:
        switch=False
    
    # assumes r x n/r x (3,4) velo
    dist = np.sqrt(velo[:, :, 0] ** 2 + velo[:, :, 1] ** 2)
    # theta = np.arctan2(velo[:, 1], velo[:, 0])
    out = np.stack([dist, velo[:, :, 2]], axis=2)
    
    if switch:
        out = out.transpose(2, 0, 1, 3)

    if len(velo.shape) == 4: 
        out = out.transpose(3, 0, 1, 2)
    
    return out

def from_polar_np(velo):
    angles = np.linspace(0, np.pi * 2, velo.shape[-1])
    dist, z = velo[:, 0], velo[:, 1]
    x = np.cos(angles) * dist
    y = np.sin(angles) * dist
    out = np.stack([x,y,z], axis=1)
    return out.astype('float32')

def getint(name):
    try:
        return int(name.split('.')[0])
    except Exception as e:
        print("Error occured while trying to read {}".format(name))
    return None

def draw_pcd(pcd, where='mat_3d'):
    if where is 'opn_nb':
        visualizer = o3d.JVisualizer()
        visualizer.add_geometry(pcd)
        visualizer.show()
    elif where is 'opn_view':
        o3d.visualization.draw_geometries([pcd], width=1280, height=800)
    elif where is 'mat_3d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1], pts[:,2])
        plt.grid()
        plt.show()
    elif where is 'mat_2d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1])
        plt.grid()
        plt.show()

In [4]:
img_list = []
for file in tqdm_notebook(sorted(os.listdir(IMG_FOLDER_PATH), key=getint)):
    img_arr = cv2.imread(os.path.join(IMG_FOLDER_PATH, file))
    img_list.append(img_arr)

/home/saby/anaconda3/envs/ati/lib/python3.6/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


1. Load image

In [5]:
file = sorted(os.listdir(IMG_FOLDER_PATH), key=getint)[10]
# img_arr = cv2.imread(os.path.join(IMG_FOLDER_PATH, file))
img_arr = cv2.imread("temp.png")
img_arr = np.concatenate([img_arr[:,:,2:3], img_arr[:,:,1:2], img_arr[:,:,0:1]], axis=-1)
img_arr.shape

(40, 512, 3)

In [6]:
temp2_arr = np.concatenate([img_arr[:,:,2:3], img_arr[:,:,1:2], img_arr[:,:,0:1]], axis=-1)
temp_arr = linear_transform(temp2_arr.transpose(2,0,1), A=0, B=255, a=-1, b=1).reshape(3,-1).T
temp_pcd = o3d.geometry.PointCloud()
temp_pcd.points = o3d.utility.Vector3dVector(temp_arr)

draw_pcd(temp_pcd, where='opn_nb')

JVisualizer with 1 geometries

In [7]:
temp_arr = img_arr.reshape(-1,3)

temp_pcd = o3d.geometry.PointCloud()
temp_pcd.points = o3d.utility.Vector3dVector(temp_arr)

draw_pcd(temp_pcd, where='opn_nb')

JVisualizer with 1 geometries

2 Transpose

In [8]:
img_transposed_arr = img_arr.transpose(2, 0, 1)
img_transposed_arr.shape

(3, 40, 512)

In [ ]:
temp_arr = img_transposed_arr.reshape(3,-1).T

temp_pcd = o3d.geometry.PointCloud()
temp_pcd.points = o3d.utility.Vector3dVector(temp_arr)

draw_pcd(temp_pcd, where='opn_nb')

3. Linear Transform

In [9]:
img_orig_arr = linear_transform(img_transposed_arr, A=0, B=255, a=-1, b=1)
img_orig_arr.shape

(3, 40, 512)

In [ ]:
temp_arr = img_orig_arr.reshape(3,-1).T

temp_pcd = o3d.geometry.PointCloud()
temp_pcd.points = o3d.utility.Vector3dVector(temp_arr)

draw_pcd(temp_pcd, where='opn_nb')

4 concatenate zeros <br>
5 transpose again <br>
6 to polar <br>
7 transpose again <br>

In [10]:
img_size = img_orig_arr.shape
npy_3d_arr = np.concatenate([img_orig_arr, np.zeros((1, img_size[1],img_size[2]))], axis=0)
npy_3d_arr.shape

(4, 40, 512)

In [11]:
npy_3d_transposed = npy_3d_arr.reshape(1,img_size[0]+1,img_size[1],img_size[2]).transpose(0,2,3,1)
npy_3d_transposed.shape

(1, 40, 512, 4)

In [12]:
npy_polar = to_polar_np(npy_3d_transposed)
npy_polar.shape

(1, 40, 512, 2)

In [13]:
npy_2d_arr = npy_polar.transpose(0, 3, 1, 2)
npy_2d_arr.shape

(1, 2, 40, 512)

In [14]:
temp_arr = from_polar_np(npy_2d_arr)[0].reshape(3,-1).T
temp_arr.shape

temp_pcd = o3d.geometry.PointCloud()
temp_pcd.points = o3d.utility.Vector3dVector(temp_arr)

draw_pcd(temp_pcd, where='opn_nb')

JVisualizer with 1 geometries

In [ ]:
img_list_arr = np.array(img_list)
img_list_arr.shape

In [ ]:
img_transposed_arr = img_list_arr.transpose(0, 3, 1, 2)
img_transposed_arr.shape

In [ ]:
img_orig_arr = linear_transform(img_transposed_arr)
img_orig_arr.shape

In [ ]:
img_size = img_orig_arr.shape
npy_3d_arr = np.concatenate([img_orig_arr, np.zeros((img_size[0],1, img_size[2],img_size[3]))], axis=1)
npy_3d_arr.shape

In [ ]:
npy_2d_arr = to_polar_np(npy_3d_arr.transpose(0,2,3,1)).transpose(0, 3, 1, 2)
npy_2d_arr.shape

In [ ]:
np.save(os.path.join("/home/saby/Projects/ati/data/data/datasets/Carla/64beam-Data/pair_transform_dynseg_all/emptycities_preprocess/test/temp", "emptycities_out.npy"), npy_2d_arr)